In [63]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')


In [64]:
df=pd.read_csv("/kaggle/input/microbial-life-in-the-great-lakes/JV Algae data 2020 complete.csv")

In [65]:
df.shape

(618, 37)

In [66]:
df=df.dropna()

In [67]:
df["sequence"][0]

'GCTTTACTGTAACTTGAAATTGGAATTGGACTTTGCTTGCGCAGTATAGGTGGGAAGCGTTGAAGATACTCTTGCGGGAGTATTGGAGCTATCAGTGAGATACCACTCTTGTAATGTTAGATTTCTAACTTTGAACCATAATCTGGTCAAAGAACAGTTTCAGGCGGGCAGTTTGACTGGGGCGGTCGCCTCCTAAATGGTAACGGAGGCGTACAAAGGTTTCCTCTGAACGGATAGAAATCGTATCTAGAGTGTAAAGGCATAAGGAAGCTTGACTGTGAGACCTACAAGTCGAGCAGGGACGAAAGTCGGTCTTAGTGATCTGACGGTACTG'

In [68]:
df.shape

(96, 37)

In [69]:
# X=df["sequence"]
Y=df[["Kingdom", "Phylum", "Class", "Order", "Family", "Genus", "Species"]]

In [70]:
Y.isnull().sum()

Kingdom    0
Phylum     0
Class      0
Order      0
Family     0
Genus      0
Species    0
dtype: int64

In [71]:
# Y=Y.fillna('0')

In [72]:
Y.nunique()

Kingdom     2
Phylum      6
Class      14
Order      23
Family     26
Genus      30
Species    40
dtype: int64

# # to  convert labels to integers# 

In [73]:
def label_converter(df):
    from sklearn.preprocessing import LabelEncoder
    label_dict={}
    encoded_labels=pd.DataFrame()
    for level in df.columns:
        label_dict[level]=LabelEncoder()
        encoded_labels[level]=label_dict[level].fit_transform(df[level])
    return label_dict,encoded_labels

label_dict,encoded_labels=label_converter(Y)


In [160]:
import pickle
with open("label_data.pkl", "wb") as f:
    pickle.dump((label_dict, encoded_labels), f)

In [166]:
with open("label_data.pkl","rb") as f:
    label_dict,encoded_labels=pickle.load(f)

In [167]:
encoded_labels

,Kingdom,Phylum,Class,Order,Family,Genus,Species
0,1,0,3,10,14,15,19
1,1,4,12,21,24,16,20
2,0,2,13,14,12,23,30
3,0,2,13,17,21,26,34
4,0,2,13,17,21,26,34
...,...,...,...,...,...,...,...
91,0,2,13,17,21,26,34
92,1,5,6,8,25,28,38
93,1,5,6,8,25,28,38
94,1,5,9,18,13,14,18


In [74]:
label_dict.values()

dict_values([LabelEncoder(), LabelEncoder(), LabelEncoder(), LabelEncoder(), LabelEncoder(), LabelEncoder(), LabelEncoder()])

In [75]:
encoded_labels

,Kingdom,Phylum,Class,Order,Family,Genus,Species
0,1,0,3,10,14,15,19
1,1,4,12,21,24,16,20
2,0,2,13,14,12,23,30
3,0,2,13,17,21,26,34
4,0,2,13,17,21,26,34
...,...,...,...,...,...,...,...
91,0,2,13,17,21,26,34
92,1,5,6,8,25,28,38
93,1,5,6,8,25,28,38
94,1,5,9,18,13,14,18


In [76]:
encoded_labels

,Kingdom,Phylum,Class,Order,Family,Genus,Species
0,1,0,3,10,14,15,19
1,1,4,12,21,24,16,20
2,0,2,13,14,12,23,30
3,0,2,13,17,21,26,34
4,0,2,13,17,21,26,34
...,...,...,...,...,...,...,...
91,0,2,13,17,21,26,34
92,1,5,6,8,25,28,38
93,1,5,6,8,25,28,38
94,1,5,9,18,13,14,18


In [77]:
# for key,value in label_dict.items():
#     print(key,value)
# # label_dict.values()

In [78]:
# X[0]

to get unique data and its index

In [79]:
# data=[]
# data_index={}
# for seq in X:
#     data.append([seq[i:i+3] for i in range(len(seq)-3+1)])
#     for ji in data:
#         for ki in ji:
#             data_index[ki]={value:i for i,value in enumerate(ji)}
    
    

# #for padding of each sequence 

In [128]:
max_length=df["sequence"].apply(len).max()
print(max_len)
def padding_seq(seq,max_length,padd_value="0"):
    return seq[:max_len] if len(seq)>max_len else seq+padd_value*(max_length-len(seq))
df["padding_seq"]=df["sequence"].apply(lambda seq:padding_seq(seq,max_length))

336


In [81]:
min_length=df["padding_seq"].apply(len).max()
print(min_length)

336


In [82]:
codons = {
    "AAA": 1, "AAC": 2, "AAG": 3, "AAT": 4,
    "ACA": 5, "ACC": 6, "ACG": 7, "ACT": 8,
    "AGA": 9, "AGC": 10, "AGG": 11, "AGT": 12,
    "ATA": 13, "ATC": 14, "ATG": 15, "ATT": 16,
    "CAA": 17, "CAC": 18, "CAG": 19, "CAT": 20,
    "CCA": 21, "CCC": 22, "CCG": 23, "CCT": 24,
    "CGA": 25, "CGC": 26, "CGG": 27, "CGT": 28,
    "CTA": 29, "CTC": 30, "CTG": 31, "CTT": 32,
    "GAA": 33, "GAC": 34, "GAG": 35, "GAT": 36,
    "GCA": 37, "GCC": 38, "GCG": 39, "GCT": 40,
    "GGA": 41, "GGC": 42, "GGG": 43, "GGT": 44,
    "GTA": 45, "GTC": 46, "GTG": 47, "GTT": 48,
    "TAA": 49, "TAC": 50, "TAG": 51, "TAT": 52,
    "TCA": 53, "TCC": 54, "TCG": 55, "TCT": 56,
    "TGA": 57, "TGC": 58, "TGG": 59, "TGT": 60,
    "TTA": 61, "TTC": 62, "TTG": 63, "TTT": 64
}


In [83]:
def num_class_in_col(df):
    # col=df.columns
    num_classes=[]
    for i in df.columns:
        
        num_classes.append(df[i].nunique())
    return num_classes
num_classes_level_orig=num_class_in_col(encoded_labels)
num_classes_level_orig

[2, 6, 14, 23, 26, 30, 40]

In [84]:
def seq_to_kmers(seq, k=3):
    return [seq[i:i+k] for i in range(0, len(seq)-k+1)]

def build_vocab(sequences, k=3):
    codons={"<PAD>": 0, "<UNK>": 1}
    idx = 2
    for seq in sequences:
        for kmer in seq_to_kmers(seq, k):
            if kmer not in codons:
                codons[kmer] = idx
                idx += 1
    return codons

In [85]:
da=build_vocab(df["sequence"])
da

{'<PAD>': 0,
 '<UNK>': 1,
 'GCT': 2,
 'CTT': 3,
 'TTT': 4,
 'TTA': 5,
 'TAC': 6,
 'ACT': 7,
 'CTG': 8,
 'TGT': 9,
 'GTA': 10,
 'TAA': 11,
 'AAC': 12,
 'TTG': 13,
 'TGA': 14,
 'GAA': 15,
 'AAA': 16,
 'AAT': 17,
 'ATT': 18,
 'TGG': 19,
 'GGA': 20,
 'GAC': 21,
 'TGC': 22,
 'GCG': 23,
 'CGC': 24,
 'GCA': 25,
 'CAG': 26,
 'AGT': 27,
 'TAT': 28,
 'ATA': 29,
 'TAG': 30,
 'AGG': 31,
 'GGT': 32,
 'GTG': 33,
 'GGG': 34,
 'AAG': 35,
 'AGC': 36,
 'CGT': 37,
 'GTT': 38,
 'AGA': 39,
 'GAT': 40,
 'CTC': 41,
 'TCT': 42,
 'CGG': 43,
 'GAG': 44,
 'CTA': 45,
 'ATC': 46,
 'TCA': 47,
 'ACC': 48,
 'CCA': 49,
 'CAC': 50,
 'ATG': 51,
 'TTC': 52,
 'CAT': 53,
 'GTC': 54,
 'CAA': 55,
 'ACA': 56,
 'GGC': 57,
 'TCG': 58,
 'GCC': 59,
 'CCT': 60,
 'TCC': 61,
 'ACG': 62,
 'CGA': 63,
 'CCC': 64,
 'CCG': 65}

In [86]:
# base_to_int={"A":1,"C":2,"G":3,"T":4,"0":0}
# for i in codons.keys():
#     try:
#     seqi=np.array([
#     [codons[seq[b:b+3]] for b in range(0,len(seq),3)] 
#     for seq in df["padding_seq"]
# ])
#     finally:
#         if codons[i] is not in seqi

# print(seqi.shape)

# seqi=np.array([
#     [codons[seq[b:b+3]] for b in range(0,len(seq),3)] 
#     for seq in df["padding_seq"]
# ])
# print(seqi)

In [87]:
X1=df["sequence"].apply(lambda x: " ".join(seq_to_kmers(x)))
X1

0      GCT CTT TTT TTA TAC ACT CTG TGT GTA TAA AAC AC...
9      GCT CTT TTT TTA TAC ACT CTG TGT GTT TTC TCC CC...
22     GCT CTT TTT TTA TAC ACT CTA TAT ATA TAG AGC GC...
27     GCT CTT TTT TTA TAC ACT CTG TGT GTA TAG AGC GC...
37     GCT CTT TTT TTA TAC ACT CTG TGT GTA TAG AGC GC...
                             ...                        
595    GCT CTT TTT TTA TAC ACT CTG TGT GTA TAG AGC GC...
606    GCT CTT TTC TCA CAC ACT CTG TGC GCA CAC ACC CC...
609    GCT CTT TTC TCA CAC ACT CTG TGC GCA CAC ACC CC...
613    GCT CTT TTT TTA TAC ACT CTG TGT GTA TAT ATC TC...
614    GCT CTT TTT TTA TAC ACT CTG TGT GTA TAG AGC GC...
Name: sequence, Length: 96, dtype: object

In [88]:
X1

0      GCT CTT TTT TTA TAC ACT CTG TGT GTA TAA AAC AC...
9      GCT CTT TTT TTA TAC ACT CTG TGT GTT TTC TCC CC...
22     GCT CTT TTT TTA TAC ACT CTA TAT ATA TAG AGC GC...
27     GCT CTT TTT TTA TAC ACT CTG TGT GTA TAG AGC GC...
37     GCT CTT TTT TTA TAC ACT CTG TGT GTA TAG AGC GC...
                             ...                        
595    GCT CTT TTT TTA TAC ACT CTG TGT GTA TAG AGC GC...
606    GCT CTT TTC TCA CAC ACT CTG TGC GCA CAC ACC CC...
609    GCT CTT TTC TCA CAC ACT CTG TGC GCA CAC ACC CC...
613    GCT CTT TTT TTA TAC ACT CTG TGT GTA TAT ATC TC...
614    GCT CTT TTT TTA TAC ACT CTG TGT GTA TAG AGC GC...
Name: sequence, Length: 96, dtype: object

In [89]:
# X=pd.DataFrame(seqi,columns=[f"{i}" for i in range(max_len)])

In [90]:
# X

In [91]:
# from keras.layers import Embedding,Input,Dense,LSTM
# from keras.models import Model
# inputi=Input(shape=(7,))
# emb=Embedding(input_dim=64,output_dim=140,input_length=350)(inputi)
# lst=LSTM(32,activation="relu",return_sequences=False)(emb)
# dens=Dense(32,activation="relu")(lst)
# output=Dense(7,activation="softmax")(dens)
# model=Model(inputi,output)
# model.compile(loss="sparse_categorical_crossentrop",optimizer="adam",metrics=["accuray"])
# model.summary()

In [92]:
# data_index

In [93]:
# keysi=data_index.keys()
# # data
# keys_index={value:i for i,value in enumerate(keysi)}
# keys_index

In [94]:
# data_index


In [95]:
# count=0
# for i in X[0]:
#     count+=1
# print(count/3)

In [96]:
encoded_labels.nunique()

Kingdom     2
Phylum      6
Class      14
Order      23
Family     26
Genus      30
Species    40
dtype: int64

In [97]:
df.iloc[:4,10:]

,% match,# species,YFQSP1CO.1,Y50IWGKC.1,XBMSFV5C.1,VNTRF25N.1,S3MA6GI2.1,R0W0YB0K.1,PQ5ZOKCT.1,PIB2ND0I.1,...,HQHKQVOO.1,F6L847VI.1,DHKAAL0M.1,D9KE4M3W.1,B38RWO34.1,9UEWYW2S.1,6OQTOL3R.1,38E9A0I2.1,0MEH7CHZ.1,padding_seq
0,100.0,1,317,155,0,603,61,221,5762,65,...,85,1317,0,61,11,64,216,14,510,GCTTTACTGTAACTTGAAATTGGAATTGGACTTTGCTTGCGCAGTA...
9,100.0,1,0,0,0,33,838,0,0,0,...,0,0,0,0,0,0,0,0,0,GCTTTACTGTTCCCTGGGATTGGGTTTGGGCCTTTTTTGCGCAGCC...
22,95.5,1,0,0,0,0,0,0,0,456,...,0,0,0,0,0,0,0,0,0,GCTTTACTATAGCCTGGAATTGGGTTCGGGCTTCGCTTGCGCAGAA...
27,98.2,1,160,0,0,0,0,0,0,0,...,240,0,0,0,0,0,0,0,0,GCTTTACTGTAGCTTGGTATTGTGTTCGGGCTCTGAATGCGCAGGA...


In [98]:
X1.shape

(96,)

In [99]:
df.columns

Index(['TestId', 'ESVId', 'sequence', 'Kingdom', 'Phylum', 'Class', 'Order',
       'Family', 'Genus', 'Species', '% match', '# species', 'YFQSP1CO.1',
       'Y50IWGKC.1', 'XBMSFV5C.1', 'VNTRF25N.1', 'S3MA6GI2.1', 'R0W0YB0K.1',
       'PQ5ZOKCT.1', 'PIB2ND0I.1', 'PFP3IAKW.1', 'P8RFOW4X.1', 'O8JZ5APC.1',
       'NIS3JHUE.1', 'NDYBOSCI.1', 'MAX9U1SY.1', 'LXNSFSRM.1', 'I2TCAGTI.1',
       'HQHKQVOO.1', 'F6L847VI.1', 'DHKAAL0M.1', 'D9KE4M3W.1', 'B38RWO34.1',
       '9UEWYW2S.1', '6OQTOL3R.1', '38E9A0I2.1', '0MEH7CHZ.1', 'padding_seq'],
      dtype='object')

In [100]:
# df.isnull().sum()

In [101]:
h=Y.nunique()
h

Kingdom     2
Phylum      6
Class      14
Order      23
Family     26
Genus      30
Species    40
dtype: int64

# #Dataset class and data loader classs 

In [102]:
from torch.utils.data import DataLoader,Dataset
from sklearn.model_selection import train_test_split
import torch
# A custom Dataset class must implement three functions: __init__, __len__, and __getitem__.
class DNA_dataset(Dataset):
    def __init__(self,sequence,labels,vocab,max_length=338):
        self.sequence=sequence
        self.labels=labels
        self.vocab=vocab
        self.max_length=max_length
    def __len__(self):
        return len(self.sequence)
    def __getitem__(self,idx):
        seq=self.sequence.iloc[idx].split()
        token_id=[self.vocab.get(token,self.vocab["<UNK>"]) for token in seq[:self.max_length]]
        token_id+=[self.vocab["<PAD>"]]*(self.max_length-len(token_id))
        label_row=self.labels.iloc[idx].values          
        label_tensor=[torch.tensor(label,dtype=torch.long) for label in label_row]
  

        return torch.tensor(token_id,dtype=torch.long), label_tensor
        
X_train,X_test,Y_train,Y_test=train_test_split(X1,encoded_labels,test_size=0.1,random_state=42)

train_dataset=DNA_dataset(X1,encoded_labels,da,max_length=338)
test_dataset=DNA_dataset(X_test,Y_test,da,max_length=338)

train_loader=DataLoader(train_dataset,batch_size=32,shuffle=True)
test_loader=DataLoader(test_dataset,batch_size=32)

# I use dataloader and Dataset lib because it helps for better training of pytorch model

In [103]:
for X_train,Y_train in train_loader:
    print(Y_train)

[tensor([1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1,
        1, 0, 1, 1, 0, 0, 0, 1]), tensor([0, 0, 0, 2, 5, 5, 5, 2, 5, 5, 2, 2, 5, 5, 0, 2, 5, 0, 5, 2, 4, 2, 0, 5,
        4, 2, 4, 4, 2, 2, 2, 0]), tensor([ 0,  0,  7, 13,  6,  6,  6, 13,  6,  6, 13, 13,  6,  6,  0, 13, 11,  7,
        11, 13, 12, 13,  0,  4, 12, 13, 12, 12, 13, 13, 13,  0]), tensor([11,  0,  7, 17,  8,  8,  8, 14,  8,  8, 14, 17,  8,  8,  0, 14, 20,  7,
        20,  3,  5, 12, 11,  4, 21, 14, 21, 21, 14, 17, 17, 11]), tensor([ 0,  1, 20, 21, 25, 25, 25, 12, 25, 25, 12, 21, 25, 25,  1, 12, 23, 20,
        23,  6,  9,  3, 15,  8, 24, 12, 24, 24, 12, 21, 21,  0]), tensor([13, 19, 17, 26, 28, 28, 28, 23, 28, 28, 23, 26, 28, 28, 19, 23, 29, 17,
        29,  3,  7,  1, 18,  8, 16, 23, 25, 16, 23, 26, 26, 13]), tensor([17, 25, 23, 34, 38, 38, 38, 30, 38, 38, 30, 34, 38, 38, 26, 30, 39, 23,
        39,  3,  8,  1, 24, 11, 20, 30, 33, 20, 30, 34, 34, 17])]
[tensor([1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0,

self.pos_encoder is a precomputed tensor of shape (1, max_length, d_model) (e.g., (1, 512, 64)) that encodes the position of each token in the sequence. These encodings are either learned (as in your model, using nn.Parameter) or fixed (e.g., sinusoidal functions in the original Transformer paper).

In [104]:
import torch 
import torch.nn as nn
num_classes_per_level = [2, 6,14,23,26,30,40]

class Transformer_model(nn.Module):
    def __init__(self,vocab_size,d_model=256,num_layers=20,nhead=8,num_classes_per_level=num_classes_level_orig):
        super(Transformer_model,self).__init__()
        self.embd=nn.Embedding(vocab_size,d_model) #d_model is dimention of each vocab_size
        self.pos_encoder=nn.Parameter(torch.zeros(1,338,d_model))
        
        self.norm1=nn.LayerNorm(d_model)
        
        encoder=nn.TransformerEncoderLayer(d_model,nhead,batch_first=True)
        self.transformer=nn.TransformerEncoder(encoder,num_layers)
        self.dropout=nn.Dropout(0.2)
        self.norm2=nn.LayerNorm(d_model)
        self.atten_pool=nn.Linear(d_model,1)
        
        self.mlp=nn.ModuleList([
        nn.Sequential(
            nn.Linear(d_model,d_model//2),
            nn.ReLU(),
            # nn.Dropout(0.2),
            nn.Linear(d_model//2,num_classes)
        )for num_classes in num_classes_per_level])
        
    def forward(self,x):
                # x.size(1) is simillar with x.shape(1)
        xi=self.embd(x)+self.pos_encoder[:,:x.size(1),:]
        xi=self.norm1(xi)
                # Each token’s embedding now includes both its semantic content (from self.embedding) and its position in the sequence (from self.pos_encoder).
        padding_mask=(x==da["<PAD>"])
        sat=self.transformer(xi,src_key_padding_mask=padding_mask)
        sat=self.norm2(sat)
        # sat=self.dropout(sat)
       
        mea=torch.softmax(self.atten_pool(sat),dim=1)
        mea=torch.sum(sat*mea,dim=1)
                # In PyTorch, the .mean(dim=1) operation calculates the mean of a tensor along the second dimension (index 1).

        outputs=[ab(mea) for ab in self.mlp]
        return outputs

optimizer.zero_grad(): Clears gradients from the previous batch.
model(batch_x): Computes predictions for a batch of tokenized DNA sequences.
criterion(outputs, batch_y): Calculates the loss based on predicted vs. actual species.
loss.backward(): Computes gradients for all model parameters.
optimizer.step(): Updates parameters to reduce the loss.

In [105]:
for col in Y.columns:
    print(col, encoded_labels[col].min(), encoded_labels[col].max())


Kingdom 0 1
Phylum 0 5
Class 0 13
Order 0 22
Family 0 25
Genus 0 29
Species 0 39


In [106]:
for key in label_dict.keys():
    print(label_dict[key])

LabelEncoder()
LabelEncoder()
LabelEncoder()
LabelEncoder()
LabelEncoder()
LabelEncoder()
LabelEncoder()


In [169]:
len(da)

66

In [ ]:
loss_entropy = nn.CrossEntropyLoss()
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device=torch.device("cpu")
model=Transformer_model(vocab_size=len(da),d_model=64,num_layers=4,nhead=8)
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=5e-4)
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=100, gamma=0.5)  # Halve lr every 10 epochs
# Here is the Epoch
epoches=500
for epoch in range(epoches):
    model.train()
    total_loss=0
    for X_train,Y_train in train_loader:
        # print(X_train.shape,Y_train.shape)
        X_train=X_train.to(device)
        Y_train=[y.to(device).long() for y in Y_train]
        optimizer.zero_grad()
        
        # Resets the gradients of all model parameters to zero.:Why it’s needed: Accumulated gradients would lead to incorrect weight updates, causing the model to learn improperly (e.g., predicting wrong species).
        
        output=model(X_train)
        loss=sum(loss_entropy(out,y) for out,y in zip(output,Y_train))
        
        # # Calculate loss gradients:Why it’s needed: Without gradients, the optimizer cannot adjust the model to improve predictions, leading to no learning.
        loss.backward()
        optimizer.step()
        # What it does: Updates the model’s parameters using the gradients computed by loss.backward().
        total_loss+=loss.item()/len(output)
        # Why it’s needed: This step is where the model learns, updating weights to minimize the loss and improve accuracy (e.g., better distinguishing species A from species B).
    # scheduler.step()
    model.eval()
    total_per_level=[0]*len(Y.columns)
    correct_pred_per_level=[0]*len(Y.columns)
    # total_loops=0
    with torch.no_grad():
    # What it does: Disables gradient computation for operations within its context, reducing memory usage and speeding up inference or evaluation.
    # Why it’s needed: For your species prediction task, this optimizes evaluation (e.g., calculating test accuracy) and inference (e.g., predicting species for new DNA sequences
        
        for X_test,Y_test in train_loader:
            X_test=X_test.to(device)
            Y_test=[y.to(device) for y in Y_test]
            outputs_test=model(X_test)
            prediction=[torch.max(output,1)[1] for output in outputs_test]
            for i,(pred,y) in enumerate(zip(prediction,Y_test)):
                total_per_level[i]+=y.size(0)
                
                correct_pred_per_level[i]+=(pred==y).sum().item() # Returns the value of this tensor as a standard Python number. This only works for tensors with one element.
            predicted_labels=[pred.cpu().numpy() for pred in prediction]
            original_labels=[label_dict[key].inverse_transform(pred) for key,pred in zip(Y.columns,predicted_labels)]
            
            # print(original_labels)
            # for level,pred in zip(Y.columns,original_labels):
                # print(level,pred)
    # accuracy_test=correct_prediction/total_loops
    # print(accuracy_test)
    accuracies = [correct / total for correct, total in zip(correct_pred_per_level, total_per_level)]
    # print(f"Validation accuracies: { accuracies))}")
    # print(f"Epoch {epoch+1}/{epoches}, Loss: {total_loss/len(train_loader), Accuracy: {accuracies}/100}")
    print(f"Epoch {epoch+1}/{epoches}, Loss: {total_loss/len(train_loader):.4f}, Accuracy: {[round(a,3) for a in accuracies]}")


In [108]:
for col in encoded_labels.columns:
    print(col, encoded_labels[col].min(), encoded_labels[col].max(), num_classes_per_level[encoded_labels.columns.get_loc(col)])


Kingdom 0 1 2
Phylum 0 5 6
Class 0 13 14
Order 0 22 23
Family 0 25 26
Genus 0 29 30
Species 0 39 40


In [109]:
xx=[y.shape for y in Y_train]
print(xx)

[torch.Size([32]), torch.Size([32]), torch.Size([32]), torch.Size([32]), torch.Size([32]), torch.Size([32]), torch.Size([32])]


In [110]:
model.eval()
total_per_level=[0]*len(Y.columns)
correct_pred_per_level=[0]*len(Y.columns)
# total_loops=0
with torch.no_grad():
# What it does: Disables gradient computation for operations within its context, reducing memory usage and speeding up inference or evaluation.
# Why it’s needed: For your species prediction task, this optimizes evaluation (e.g., calculating test accuracy) and inference (e.g., predicting species for new DNA sequences
    
    for X_test,Y_test in test_loader:
        X_test=X_test.to(device)
        Y_test=[y.to(device) for y in Y_test]
        outputs_test=model(X_test)
        prediction=[torch.max(output,1)[1] for output in outputs_test]
        for i,(pred,y) in enumerate(zip(prediction,Y_test)):
            total_per_level[i]+=y.size(0)
            
            correct_pred_per_level[i]+=(pred==y).sum().item() # Returns the value of this tensor as a standard Python number. This only works for tensors with one element.
        predicted_labels=[pred.cpu().numpy() for pred in prediction]
        original_labels=[label_dict[key].inverse_transform(pred) for key,pred in zip(label_dict.keys(),predicted_labels)]
        
        # print(original_labels)
        # for level,pred in zip(Y.columns,original_labels):
            # print(level,pred)
# accuracy_test=correct_prediction/total_loops
# print(accuracy_test)
accuracies = [correct / total for correct, total in zip(correct_pred_per_level, total_per_level)]
print(f"Validation accuracies: {dict(zip(Y.columns, accuracies))}")

Validation accuracies: {'Kingdom': 1.0, 'Phylum': 1.0, 'Class': 1.0, 'Order': 0.9, 'Family': 1.0, 'Genus': 0.9, 'Species': 0.9}


In [168]:
def predict_taxon(model_path,single_sequence):
    model=Transformer_model(vocab_size=len(da),d_model=64,num_layers=4,nhead=8,num_classes_per_level=num_classes_per_level).to(device)

    model.load_state_dict(torch.load(model_path,map_location=device))
    
    model.eval()
    
    # Example inference
    new_sequence = single_sequence
    kmer_seq = " ".join(seq_to_kmers(new_sequence))
    # vocab=build_vocab(df["sequence"])
    # seq=self.sequence.iloc[idx].split()
    token_id=[da.get(token,"<UNK>") for token in kmer_seq.split()[:max_length]]
    token_id+=[da["<PAD>"]]*(max_length-len(token_id))
    
    inputs = torch.tensor(token_id,dtype=torch.long).unsqueeze(0).to(device)
    with torch.no_grad():
        outputs = model(inputs)
        predicted_indices = [torch.argmax(out, dim=1).item() for out in outputs]  # Get index for each level
    
        predicted_spec= [label_dict[level].inverse_transform([output])[0] for level ,output  in zip(label_dict.keys(), predicted_indices)]
    for level,pred in zip(Y.columns,predicted_spec):
        print(f"{level}:{pred}")
    # print(f"Predicted species: {predicted_species[0]}")
predict_taxon("our_model500.pth",df["sequence"].iloc[0])

Kingdom:Eukaryota
Phylum:Bacillariophyta
Class:Coscinodiscophyceae
Order:Melosirales
Family:Melosiraceae
Genus:Melosira
Species:Melosira varians


In [131]:
df

,TestId,ESVId,sequence,Kingdom,Phylum,Class,Order,Family,Genus,Species,...,HQHKQVOO.1,F6L847VI.1,DHKAAL0M.1,D9KE4M3W.1,B38RWO34.1,9UEWYW2S.1,6OQTOL3R.1,38E9A0I2.1,0MEH7CHZ.1,padding_seq
0,23S,ESV_001015,GCTTTACTGTAACTTGAAATTGGAATTGGACTTTGCTTGCGCAGTA...,Eukaryota,Bacillariophyta,Coscinodiscophyceae,Melosirales,Melosiraceae,Melosira,Melosira varians,...,85,1317,0,61,11,64,216,14,510,GCTTTACTGTAACTTGAAATTGGAATTGGACTTTGCTTGCGCAGTA...
9,23S,ESV_063854,GCTTTACTGTTCCCTGGGATTGGGTTTGGGCCTTTTTTGCGCAGCC...,Eukaryota,Streptophyta,Zygnemophyceae,Zygnematales,Zygnemataceae,Mougeotia,Mougeotia sp. 2 AS-2014,...,0,0,0,0,0,0,0,0,0,GCTTTACTGTTCCCTGGGATTGGGTTTGGGCCTTTTTTGCGCAGCC...
22,23S,ESV_062483,GCTTTACTATAGCCTGGAATTGGGTTCGGGCTTCGCTTGCGCAGAA...,Bacteria,Cyanobacteria,unk_class,Pleurocapsales,Hyellaceae,Pleurocapsa,Pleurocapsa sp. 04302-00001,...,0,0,0,0,0,0,0,0,0,GCTTTACTATAGCCTGGAATTGGGTTCGGGCTTCGCTTGCGCAGAA...
27,23S,ESV_061884,GCTTTACTGTAGCTTGGTATTGTGTTCGGGCTCTGAATGCGCAGGA...,Bacteria,Cyanobacteria,unk_class,Synechococcales,Synechococcaceae,Synechococcus,Synechococcus lacustris,...,240,0,0,0,0,0,0,0,0,GCTTTACTGTAGCTTGGTATTGTGTTCGGGCTCTGAATGCGCAGGA...
37,23S,ESV_036765,GCTTTACTGTAGCTTGATATTGTGTTCGGGCTCTGAATGCGCAGGA...,Bacteria,Cyanobacteria,unk_class,Synechococcales,Synechococcaceae,Synechococcus,Synechococcus lacustris,...,186,0,0,0,0,0,13,0,0,GCTTTACTGTAGCTTGATATTGTGTTCGGGCTCTGAATGCGCAGGA...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,23S,ESV_102967,GCTTTACTGTAGCTTGGTATTGTGTTCGGGCTCTGAATGCGCAGGA...,Bacteria,Cyanobacteria,unk_class,Synechococcales,Synechococcaceae,Synechococcus,Synechococcus lacustris,...,8,0,0,0,0,0,0,0,0,GCTTTACTGTAGCTTGGTATTGTGTTCGGGCTCTGAATGCGCAGGA...
606,23S,ESV_088392,GCTTCACTGCACCCTGGGATTGAACCTGGTCTTTAGTTGCGCAGCA...,Eukaryota,unk_phylum,Eustigmatophyceae,Goniochloridales,unk_family,Trachydiscus,Trachydiscus minutus,...,8,0,0,0,0,0,0,0,0,GCTTCACTGCACCCTGGGATTGAACCTGGTCTTTAGTTGCGCAGCA...
609,23S,ESV_071698,GCTTCACTGCACCCTGGGATTGAACCTGGTCTTTAGTTGCGCAGCA...,Eukaryota,unk_phylum,Eustigmatophyceae,Goniochloridales,unk_family,Trachydiscus,Trachydiscus minutus,...,0,0,8,0,0,0,0,0,0,GCTTCACTGCACCCTGGGATTGAACCTGGTCTTTAGTTGCGCAGCA...
613,23S,ESV_064268,GCTTTACTGTATCTTGATATTGGATTTTGATTTTATTTGTGCAGAA...,Eukaryota,unk_phylum,Synurophyceae,Synurales,Mallomonadaceae,Mallomonas,Mallomonas oviformis,...,0,0,0,0,0,0,8,0,0,GCTTTACTGTATCTTGATATTGGATTTTGATTTTATTTGTGCAGAA...


In [111]:
torch.save(model.state_dict(),"our_model500.pth") # to save the model we train
# to load the saved model
# model.load_state_dict(torch.load("our_model.pth"))

# #Transfer Learning 

In [133]:
# load different datasets at a time 
dirty="/kaggle/input/microbial-life-in-the-great-lakes/JVB1879-ArthCOI-read-data.csv"
# for dy in os.listdir(dirty):
#     path=os.path.join(dirty,dy)
#     df=pd.read_csv(path)
df2=pd.read_csv("/kaggle/input/microbial-life-in-the-great-lakes/JV Fish data 2020 complete.csv")

df2=df2.dropna(how="any")
print(df2.shape)
df2.isnull().sum()

(79, 37)


TestId        0
ESVId         0
sequence      0
Kingdom       0
Phylum        0
Class         0
Order         0
Family        0
Genus         0
Species       0
% match       0
# species     0
YFQSP1CO.1    0
Y50IWGKC.1    0
XBMSFV5C.1    0
VNTRF25N.1    0
S3MA6GI2.1    0
R0W0YB0K.1    0
PQ5ZOKCT.1    0
PIB2ND0I.1    0
PFP3IAKW.1    0
P8RFOW4X.1    0
O8JZ5APC.1    0
NIS3JHUE.1    0
NDYBOSCI.1    0
MAX9U1SY.1    0
LXNSFSRM.1    0
I2TCAGTI.1    0
HQHKQVOO.1    0
F6L847VI.1    0
DHKAAL0M.1    0
D9KE4M3W.1    0
B38RWO34.1    0
9UEWYW2S.1    0
6OQTOL3R.1    0
38E9A0I2.1    0
0MEH7CHZ.1    0
dtype: int64

In [135]:
Y2=df2[["Kingdom", "Phylum", "Class", "Order", "Family", "Genus", "Species"]]
Y2

,Kingdom,Phylum,Class,Order,Family,Genus,Species
1,Eukaryota,Chordata,Actinopteri,Cypriniformes,Catostomidae,Deltistes,Deltistes luxatus
2,Eukaryota,Chordata,Actinopteri,Salmoniformes,Salmonidae,Salmo,Salmo trutta
3,Eukaryota,Chordata,Actinopteri,Cypriniformes,Leuciscidae,Lavinia,Lavinia exilicauda
4,Eukaryota,Chordata,Actinopteri,Centrarchiformes,Centrarchidae,Lepomis,Lepomis cyanellus
10,Eukaryota,Chordata,Actinopteri,Cyprinodontiformes,Poeciliidae,Gambusia,Gambusia affinis
...,...,...,...,...,...,...,...
222,Eukaryota,Chordata,Actinopteri,Siluriformes,Ictaluridae,Ameiurus,Ameiurus catus
225,Eukaryota,Chordata,Actinopteri,Cypriniformes,Catostomidae,Deltistes,Deltistes luxatus
226,Eukaryota,Chordata,Actinopteri,Cypriniformes,Catostomidae,Deltistes,Deltistes luxatus
236,Eukaryota,Chordata,Actinopteri,Cypriniformes,Catostomidae,Deltistes,Deltistes luxatus


In [136]:
Y2.nunique()

Kingdom     1
Phylum      1
Class       1
Order       5
Family      6
Genus      13
Species    19
dtype: int64

In [137]:
label_dict2,encoded_labels2=label_converter(Y2)

In [138]:
encoded_labels2

,Kingdom,Phylum,Class,Order,Family,Genus,Species
0,0,0,0,1,0,2,4
1,0,0,0,3,5,11,17
2,0,0,0,1,3,5,7
3,0,0,0,0,1,6,8
4,0,0,0,2,4,3,5
...,...,...,...,...,...,...,...
74,0,0,0,4,2,0,0
75,0,0,0,1,0,2,4
76,0,0,0,1,0,2,4
77,0,0,0,1,0,2,4


In [139]:
max_length2=df2["sequence"].apply(len).max()
print(max_length2)
# def padding_seq(seq,max_len,padd_value="0"):
#     return seq[:max_len] if len(seq)>max_len else seq+padd_value*(max_len-len(seq))
# df2["padding_seq"]=df2["sequence"].apply(lambda seq:padding_seq(seq,max_len))

176


In [140]:
vocab_size2=build_vocab(df2["sequence"])
len(vocab_size2)

66

In [141]:
# vocab_size2

In [142]:
X2=df2["sequence"].apply(lambda x: " ".join(seq_to_kmers(x)))
X2

1      CAC ACC CCG CGC GCG CGG GGT GTT TTA TAT ATA TA...
2      CAC ACC CCG CGC GCG CGG GGT GTT TTA TAT ATA TA...
3      CAC ACC CCG CGC GCG CGG GGT GTT TTA TAA AAA AA...
4      CAC ACC CCG CGC GCG CGG GGT GTT TTA TAT ATA TA...
10     CAC ACC CCG CGC GCG CGG GGT GTT TTA TAT ATA TA...
                             ...                        
222    CAC ACC CCG CGC GCG CGG GGT GTT TTA TAT ATA TA...
225    CAC ACC CCG CGC GCG CGG GGT GTT TTA TAA AAA AA...
226    CAC ACC CCG CGC GCG CGG GGT GTT TTA TAA AAA AA...
236    CAC ACC CCG CGC GCG CGG GGT GTT TTA TAT ATA TA...
237    CAC ACC CCG CGC GCG CGG GGT GTT TTA TAT ATA TA...
Name: sequence, Length: 79, dtype: object

In [143]:
Y2.columns

Index(['Kingdom', 'Phylum', 'Class', 'Order', 'Family', 'Genus', 'Species'], dtype='object')

In [144]:
Y2['Species'].nunique()

19

# # to ffunction to determine no of classes in each column


In [145]:

num_classes_per_level2=num_class_in_col(Y2)
num_classes_per_level2

[1, 1, 1, 5, 6, 13, 19]

In [152]:
train_dataset2=DNA_dataset(df2["sequence"],encoded_labels2,vocab_size2,max_length=max_length2)

train_loader2=DataLoader(train_dataset2,batch_size=32,shuffle=True)
def transfer_learning_model(data_loader,vocab_size,num_classes_per_level2,save_path="transfer_our_model3500.pth"):
    
    model=Transformer_model(vocab_size=vocab_size,d_model=64,num_layers=4,nhead=8,num_classes_per_level=num_classes_per_level2)
    pretrained_state=torch.load("our_model500.pth",map_location="cpu")
    model_state=model.state_dict()
    compatible_key=[k for k in pretrained_state.keys() if k.startswith(("embd","pos_encoder","transformer","norm1","norm2"))]
                    
    # model.load_state_dict(torch.load("our_model2500.pth"),strict=False)
    for key in compatible_key:
        if key in model_state:
            model_state[key].copy_(pretrained_state[key])
    model.load_state_dict(model_state)

    loss_entropy = nn.CrossEntropyLoss()
    device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
    for i in range(model.transformer.num_layers-2):
        
        for param in model.transformer.layers[i].parameters():
            param.requires_grad=False
    model.embd.weight.requires_grad=False
    model.to(device)


    optimizer = torch.optim.Adam(model.parameters(), lr=5e-4)
    # scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=100, gamma=0.5)  # Halve lr every 10 epochs
    # Here is the Epoch
    epoches=500
    for epoch in range(epoches):
        model.train()
        total_loss=0
        for X_train,Y_train in train_loader2:
            # print(X_train.shape,Y_train.shape)
            X_train=X_train.to(device)
            Y_train=[y.to(device).long() for y in Y_train]
            optimizer.zero_grad()
            
            # Resets the gradients of all model parameters to zero.:Why it’s needed: Accumulated gradients would lead to incorrect weight updates, causing the model to learn improperly (e.g., predicting wrong species).
            
            output=model(X_train)
            loss=sum(loss_entropy(out,y) for out,y in zip(output,Y_train))
            
            # # Calculate loss gradients:Why it’s needed: Without gradients, the optimizer cannot adjust the model to improve predictions, leading to no learning.
            loss.backward()
            optimizer.step()
            # What it does: Updates the model’s parameters using the gradients computed by loss.backward().
            total_loss+=loss.item()/len(output)
            # Why it’s needed: This step is where the model learns, updating weights to minimize the loss and improve accuracy (e.g., better distinguishing species A from species B).
        # scheduler.step()
        model.eval()
        total_per_level=[0]*len(Y.columns)
        correct_pred_per_level=[0]*len(Y.columns)
        # total_loops=0
        with torch.no_grad():
        # What it does: Disables gradient computation for operations within its context, reducing memory usage and speeding up inference or evaluation.
        # Why it’s needed: For your species prediction task, this optimizes evaluation (e.g., calculating test accuracy) and inference (e.g., predicting species for new DNA sequences
            
            for X_test,Y_test in train_loader2:
                X_test=X_test.to(device)
                Y_test=[y.to(device) for y in Y_test]
                outputs_test=model(X_test)
                prediction=[torch.max(output,1)[1] for output in outputs_test]
                for i,(pred,y) in enumerate(zip(prediction,Y_test)):
                    total_per_level[i]+=y.size(0)
                    
                    correct_pred_per_level[i]+=(pred==y).sum().item() # Returns the value of this tensor as a standard Python number. This only works for tensors with one element.
                predicted_labels=[pred.cpu().numpy() for pred in prediction]
                original_labels=[label_dict[key].inverse_transform(pred) for key,pred in zip(label_dict2.keys(),predicted_labels)]
                
                # print(original_labels)
                # for level,pred in zip(Y.columns,original_labels):
                    # print(level,pred)
        # accuracy_test=correct_prediction/total_loops
        # print(accuracy_test)
        accuracies = [correct / total for correct, total in zip(correct_pred_per_level, total_per_level)]
        # print(f"Validation accuracies: { accuracies))}")
        # print(f"Epoch {epoch+1}/{epoches}, Loss: {total_loss/len(train_loader), Accuracy: {accuracies}/100}")
        print(f"Epoch {epoch+1}/{epoches}, Loss: {total_loss/len(train_loader):.4f}, Accuracy: {[round(a,3) for a in accuracies]}")

    torch.save(model.state_dict(), save_path)
    print(f"Model saved to {save_path}")
    return 
transfer_learning_model(train_loader2,len(vocab_size2),num_classes_per_level2)

Epoch 1/500, Loss: 1.3821, Accuracy: [1.0, 1.0, 1.0, 0.152, 0.114, 0.013, 0.013]
Epoch 2/500, Loss: 1.2771, Accuracy: [1.0, 1.0, 1.0, 0.152, 0.114, 0.013, 0.013]
Epoch 3/500, Loss: 1.2172, Accuracy: [1.0, 1.0, 1.0, 0.152, 0.304, 0.013, 0.013]
Epoch 4/500, Loss: 1.1761, Accuracy: [1.0, 1.0, 1.0, 0.152, 0.304, 0.013, 0.013]
Epoch 5/500, Loss: 1.1380, Accuracy: [1.0, 1.0, 1.0, 0.152, 0.304, 0.013, 0.013]
Epoch 6/500, Loss: 1.1252, Accuracy: [1.0, 1.0, 1.0, 0.696, 0.304, 0.013, 0.013]
Epoch 7/500, Loss: 1.0969, Accuracy: [1.0, 1.0, 1.0, 0.696, 0.304, 0.025, 0.013]
Epoch 8/500, Loss: 1.0821, Accuracy: [1.0, 1.0, 1.0, 0.696, 0.304, 0.025, 0.013]
Epoch 9/500, Loss: 1.0320, Accuracy: [1.0, 1.0, 1.0, 0.696, 0.304, 0.025, 0.013]
Epoch 10/500, Loss: 1.0162, Accuracy: [1.0, 1.0, 1.0, 0.696, 0.304, 0.025, 0.013]
Epoch 11/500, Loss: 1.0186, Accuracy: [1.0, 1.0, 1.0, 0.696, 0.304, 0.025, 0.013]
Epoch 12/500, Loss: 0.9983, Accuracy: [1.0, 1.0, 1.0, 0.696, 0.304, 0.025, 0.013]
Epoch 13/500, Loss: 0.980

# # to make prediction from a model# 

In [156]:
def predict_taxon(model_path,single_sequence):
    model=Transformer_model(vocab_size=len(da),d_model=64,num_layers=4,nhead=8,num_classes_per_level=num_classes_per_level2).to(device)

    model.load_state_dict(torch.load(model_path,map_location=device))
    
    model.eval()
    
    # Example inference
    new_sequence = single_sequence
    kmer_seq = " ".join(seq_to_kmers(new_sequence))
    # vocab=build_vocab(df["sequence"])
    # seq=self.sequence.iloc[idx].split()
    token_id=[da.get(token,"<UNK>") for token in kmer_seq.split()[:max_length2]]
    token_id+=[da["<PAD>"]]*(max_length2-len(token_id))
    
    inputs = torch.tensor(token_id,dtype=torch.long).unsqueeze(0).to(device)
    with torch.no_grad():
        outputs = model(inputs)
        predicted_indices = [torch.argmax(out, dim=1).item() for out in outputs]  # Get index for each level
    
        predicted_spec= [label_dict2[level].inverse_transform([output])[0] for level ,output  in zip(label_dict2.keys(), predicted_indices)]
    for level,pred in zip(Y2.columns,predicted_spec):
        print(f"{level}:{pred}")
    # print(f"Predicted species: {predicted_species[0]}")
predict_taxon("/kaggle/working/transfer_our_model3500.pth",df2["sequence"].iloc[0])

Kingdom:Eukaryota
Phylum:Chordata
Class:Actinopteri
Order:Cypriniformes
Family:Catostomidae
Genus:Deltistes
Species:Deltistes luxatus


In [157]:
df2

,TestId,ESVId,sequence,Kingdom,Phylum,Class,Order,Family,Genus,Species,...,I2TCAGTI.1,HQHKQVOO.1,F6L847VI.1,DHKAAL0M.1,D9KE4M3W.1,B38RWO34.1,9UEWYW2S.1,6OQTOL3R.1,38E9A0I2.1,0MEH7CHZ.1
1,MiFishU,ESV_014423,CACCGCGGTTATACGAGAGGCCCTAGTTGATAGGCACGGCGTAAAG...,Eukaryota,Chordata,Actinopteri,Cypriniformes,Catostomidae,Deltistes,Deltistes luxatus,...,1133,0,0,93,0,0,30567,1292,0,42508
2,MiFishU,ESV_000043,CACCGCGGTTATACGAGAGACCCTAGTTGATAACTACCGGCGTAAA...,Eukaryota,Chordata,Actinopteri,Salmoniformes,Salmonidae,Salmo,Salmo trutta,...,0,355,62549,0,34,437,0,36,320,0
3,MiFishU,ESV_015983,CACCGCGGTTAAACGAGAGGCCCTAGTTGATAATACAACGGCGTAA...,Eukaryota,Chordata,Actinopteri,Cypriniformes,Leuciscidae,Lavinia,Lavinia exilicauda,...,7631,0,0,10403,0,0,0,0,0,0
4,MiFishU,ESV_001124,CACCGCGGTTATACGAGAGGCTCAAGTTGATAAACCCCGGCGTAAA...,Eukaryota,Chordata,Actinopteri,Centrarchiformes,Centrarchidae,Lepomis,Lepomis cyanellus,...,0,0,177,0,0,0,32565,86,0,0
10,MiFishU,ESV_009594,CACCGCGGTTATACGAGAGGCCCAAGTTGATAAAATACGGCGTAAA...,Eukaryota,Chordata,Actinopteri,Cyprinodontiformes,Poeciliidae,Gambusia,Gambusia affinis,...,0,187,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222,MiFishU,ESV_015098,CACCGCGGTTATACGAGAGGCCCTAGTTGATAGGCACGGCGTAAAG...,Eukaryota,Chordata,Actinopteri,Siluriformes,Ictaluridae,Ameiurus,Ameiurus catus,...,0,0,0,0,0,0,9,0,0,0
225,MiFishU,ESV_017105,CACCGCGGTTAAACGAGAGGCCCTAGTTGATAGGCACGGCGTAAAG...,Eukaryota,Chordata,Actinopteri,Cypriniformes,Catostomidae,Deltistes,Deltistes luxatus,...,8,0,0,0,0,0,0,0,0,0
226,MiFishU,ESV_017113,CACCGCGGTTAAACGAGAGGCCCTAGTTGATAGGCACGGCGTAAAG...,Eukaryota,Chordata,Actinopteri,Cypriniformes,Catostomidae,Deltistes,Deltistes luxatus,...,8,0,0,0,0,0,0,0,0,0
236,MiFishU,ESV_015176,CACCGCGGTTATACGAGAGGCCCTAGTTGATAGGCACGGCGTAAAG...,Eukaryota,Chordata,Actinopteri,Cypriniformes,Catostomidae,Deltistes,Deltistes luxatus,...,0,0,0,0,0,0,0,0,0,8


In [148]:
# /kaggle/input/microbial-life-in-the-great-lakes/JVB1227-MiFishU-esv-data.csv" less null value


In [149]:
df2

,TestId,ESVId,sequence,Kingdom,Phylum,Class,Order,Family,Genus,Species,...,I2TCAGTI.1,HQHKQVOO.1,F6L847VI.1,DHKAAL0M.1,D9KE4M3W.1,B38RWO34.1,9UEWYW2S.1,6OQTOL3R.1,38E9A0I2.1,0MEH7CHZ.1
1,MiFishU,ESV_014423,CACCGCGGTTATACGAGAGGCCCTAGTTGATAGGCACGGCGTAAAG...,Eukaryota,Chordata,Actinopteri,Cypriniformes,Catostomidae,Deltistes,Deltistes luxatus,...,1133,0,0,93,0,0,30567,1292,0,42508
2,MiFishU,ESV_000043,CACCGCGGTTATACGAGAGACCCTAGTTGATAACTACCGGCGTAAA...,Eukaryota,Chordata,Actinopteri,Salmoniformes,Salmonidae,Salmo,Salmo trutta,...,0,355,62549,0,34,437,0,36,320,0
3,MiFishU,ESV_015983,CACCGCGGTTAAACGAGAGGCCCTAGTTGATAATACAACGGCGTAA...,Eukaryota,Chordata,Actinopteri,Cypriniformes,Leuciscidae,Lavinia,Lavinia exilicauda,...,7631,0,0,10403,0,0,0,0,0,0
4,MiFishU,ESV_001124,CACCGCGGTTATACGAGAGGCTCAAGTTGATAAACCCCGGCGTAAA...,Eukaryota,Chordata,Actinopteri,Centrarchiformes,Centrarchidae,Lepomis,Lepomis cyanellus,...,0,0,177,0,0,0,32565,86,0,0
10,MiFishU,ESV_009594,CACCGCGGTTATACGAGAGGCCCAAGTTGATAAAATACGGCGTAAA...,Eukaryota,Chordata,Actinopteri,Cyprinodontiformes,Poeciliidae,Gambusia,Gambusia affinis,...,0,187,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222,MiFishU,ESV_015098,CACCGCGGTTATACGAGAGGCCCTAGTTGATAGGCACGGCGTAAAG...,Eukaryota,Chordata,Actinopteri,Siluriformes,Ictaluridae,Ameiurus,Ameiurus catus,...,0,0,0,0,0,0,9,0,0,0
225,MiFishU,ESV_017105,CACCGCGGTTAAACGAGAGGCCCTAGTTGATAGGCACGGCGTAAAG...,Eukaryota,Chordata,Actinopteri,Cypriniformes,Catostomidae,Deltistes,Deltistes luxatus,...,8,0,0,0,0,0,0,0,0,0
226,MiFishU,ESV_017113,CACCGCGGTTAAACGAGAGGCCCTAGTTGATAGGCACGGCGTAAAG...,Eukaryota,Chordata,Actinopteri,Cypriniformes,Catostomidae,Deltistes,Deltistes luxatus,...,8,0,0,0,0,0,0,0,0,0
236,MiFishU,ESV_015176,CACCGCGGTTATACGAGAGGCCCTAGTTGATAGGCACGGCGTAAAG...,Eukaryota,Chordata,Actinopteri,Cypriniformes,Catostomidae,Deltistes,Deltistes luxatus,...,0,0,0,0,0,0,0,0,0,8
